<div style="background-color:#f4f8ff; padding:16px; border-left:6px solid #1f4fd8; border-radius:6px; color:#000;">

<h2 style="margin-top:0; color:#000;">Performance Analysis (Year-over-Year, Month-over-Month)</h2>

<h4 style="color:#000;">Purpose</h4>
<ul>
  <li>Measure the performance of products, customers, or regions over time.</li>
  <li>Support benchmarking and identification of high-performing entities.</li>
  <li>Track year-over-year and month-over-month trends and growth.</li>
</ul>

<h4 style="color:#000;">SQL Functions Used</h4>
<ul>
  <li><b>LAG()</b> – Accesses values from previous time periods.</li>
  <li><b>AVG() OVER()</b> – Computes average values across defined partitions.</li>
  <li><b>CASE</b> – Applies conditional logic for trend and performance classification.</li>
</ul>

</div>


<div style="background-color:#f4f8ff; padding:10px; border-left:6px solid #1f4fd8; border-radius:6px; color:#000; font-size:20px;">
  <b>1. Analyze the yearly performance of products by comparing their sales with both the product’s average sales performance and the previous year’s sales.</b><br>
  <span style="font-size:16px;"></span>
</div>

In [2]:
query = """
WITH yearly_product_sales AS (
    SELECT
        YEAR(f.order_date) AS order_year,
        p.product_name,
        SUM(f.sales_amount) AS current_sales
    FROM gold.fact_sales f
    LEFT JOIN gold.dim_products p
        ON f.product_key = p.product_key
    WHERE f.order_date IS NOT NULL or f.sales_amount is not null
    GROUP BY 
        YEAR(f.order_date),
        p.product_name

)
SELECT
    order_year,
    product_name,
    current_sales,
    AVG(current_sales) OVER (PARTITION BY product_name) AS avg_sales,
    current_sales - AVG(current_sales) OVER (PARTITION BY product_name) AS diff_avg,
    CASE 
        WHEN current_sales - AVG(current_sales) OVER (PARTITION BY product_name) > 0 THEN 'Above Avg'
        WHEN current_sales - AVG(current_sales) OVER (PARTITION BY product_name) < 0 THEN 'Below Avg'
        ELSE 'Avg'
    END AS avg_change,
    -- Year-over-Year Analysis
    LAG(current_sales) OVER (PARTITION BY product_name ORDER BY order_year) AS py_sales,
    current_sales - LAG(current_sales) OVER (PARTITION BY product_name ORDER BY order_year) AS diff_py,
    CASE 
        WHEN current_sales - LAG(current_sales) OVER (PARTITION BY product_name ORDER BY order_year) > 0 THEN 'Increase'
        WHEN current_sales - LAG(current_sales) OVER (PARTITION BY product_name ORDER BY order_year) < 0 THEN 'Decrease'
        ELSE 'No Change'
    END AS py_change
FROM yearly_product_sales
ORDER BY product_name, order_year;

"""

df = pd.read_sql(query, engine)
display(HTML(df.to_html(index=False)))

order_year,product_name,current_sales,avg_sales,diff_avg,avg_change,py_sales,diff_py,py_change
2012.0,All-Purpose Bike Stand,159,13197,-13038,Below Avg,NaN,NaN,No Change
2013.0,All-Purpose Bike Stand,37683,13197,24486,Above Avg,159.0,37524.0,Increase
2014.0,All-Purpose Bike Stand,1749,13197,-11448,Below Avg,37683.0,-35934.0,Decrease
2012.0,AWC Logo Cap,72,6570,-6498,Below Avg,NaN,NaN,No Change
2013.0,AWC Logo Cap,18891,6570,12321,Above Avg,72.0,18819.0,Increase
2014.0,AWC Logo Cap,747,6570,-5823,Below Avg,18891.0,-18144.0,Decrease
2013.0,Bike Wash - Dissolver,6960,3636,3324,Above Avg,NaN,NaN,No Change
2014.0,Bike Wash - Dissolver,312,3636,-3324,Below Avg,6960.0,-6648.0,Decrease
2013.0,Classic Vest- L,11968,6240,5728,Above Avg,NaN,NaN,No Change
2014.0,Classic Vest- L,512,6240,-5728,Below Avg,11968.0,-11456.0,Decrease
